<span style="font-size:110%; text-align:left;"><b><a href="../index.html">3. Gaussian Beams</a> > <a href="./index.html">2. Photodetectors</a></b></span>
<br/><br/>
<span style="font-size:200%; text-align:left;"><b>Beat coefficients</b></span>
<br/><br/>
<span style="font-size:110%; text-align:left;">Author: Daniel Brown </span>

# Table of Contents
 <p><div class="lev1"><a href="#Composite-photo-detectors"><span class="toc-item-num">1&nbsp;&nbsp;</span>Composite photo detectors</a></div><div class="lev1"><a href="#Split-photo-detectors"><span class="toc-item-num">2&nbsp;&nbsp;</span>Split photo detectors</a></div><div class="lev1"><a href="#Bullseye-detectors"><span class="toc-item-num">3&nbsp;&nbsp;</span>Bullseye detectors</a></div>

# Composite photo detectors

If Gaussian modes are detected by a simple photo detector which is much larger than the beam size, the total power is the sum of the powers in each modes because the modes are orthogonal.

However, if the photo detector has some features (or is to small) the modes couple to each other so that cross terms must be computed. This feature is used in so-called composite photo detectors that consist of seprate elements whose signals is are added or subtracted to generate the detector output signal. 

# Split photo detectors

The ideal split photo detector consists of two half-infinite planes. The signal from the two planes are subtracte from each other. For a beam which is centered on the line seperating the two planes, the detector will only show the beat between symmetric and asymmetric modes, which is useful for generating alignment signals.

For a beam: $$E~=~~\sum_{n,m}~a_{nm}~u_{nm}(x,y,z)$$
the power is proportional to $\int E*E^*$

For a normal detector the integral would be over the entire x-y plane. For the split-photodectors (here assuming split along the vertical y-axis) we get
$$P\sim \int_0^\infty E*E^* - \int_0^{-\infty} E*E^* =2 \int_0^\infty E*E^*$$

In order to simplify the calculation we can combute so called beat coefficents $c_{n1,n2}$ so that we can write
$$P~\sim~\sum_{m}\sum_{n1,n2}~ c_{n1,n2} ~a_{n1,m}~a_{n2,m}^*$$
These can be computed as follows:

In [1]:
import numpy as np
import math
from math import factorial
from scipy.integrate import quad
from scipy.special import hermite

In [2]:
def HG_split_diode_coefficient_numerical(n1,n2):
    # return zero if n1,n2 are both even or both odd
    if (n1+n2) % 2 == 0:
        return 0.0
    # extract normlisation coefficients
    A = 2.0 * np.sqrt(2.0/np.pi) * np.sqrt(1.0 / (2.0**(n1+n2) * factorial(n1) * factorial(n2)))
    # define rest of u_n1 * u_n2^* as integrand
    f = lambda x: hermite(n1)(np.sqrt(2.0)*x) * math.exp(-2.0*x*x) * hermite(n2)(np.sqrt(2.0)*x)    
    # perform numerical integration
    val, res= quad(f, 0.0, np.Inf, epsabs=1e-10, epsrel=1e-10, limit=200)
    return A * val

In [3]:
print(HG_split_diode_coefficient_numerical(0,0))
print(HG_split_diode_coefficient_numerical(0,1))

0.0
0.7978845608028656


If both n1 and n2 are even or odd then the beat coefficient is zero, if one is odd and one even, the value is computed by numerical integration.

This function has been implemented in PyKat and can be used as follows:

In [4]:
import pykat.optics.pdtype as pdtype
pdtype.HG_split_diode_coefficient_numerical(0,1)

                                              ..-
    PyKat 1.1.349         _                  '(
                          \`.|\.__...-""""-_." )
       ..+-----.._        /  ' `            .-'
   . '            `:      7/* _/._\    \   (
  (        '::;;+;;:      `-"' =" /,`"" `) /
  L.        \`:::a:f            c_/     n_'
  ..`--...___`.  .    ,
   `^-....____:   +.      www.gwoptics.org/pykat



0.7978845608028656

The beat coefficients can be used to define a new photodiode type with the 'pdtype' command and the respective entry in the kat.ini file. To creat the coefficients in the format for the kat.ini file please use
pdtype.finesse_split_photodiode(maxtem,'x' or 'y'), for example: 

In [5]:
pdtype.finesse_split_photodiode(5,'x') 

PDTYPE x-split
 0 x  1 x 0.797884560802866
 0 x  3 x -0.32573500793528
 0 x  5 x 0.218509686118416
 2 x  1 x 0.564189583547757
 2 x  3 x 0.690988298942672
 2 x  5 x -0.257516134682126
 4 x  1 x -0.16286750396764
 4 x  3 x 0.59841342060215
 4 x  5 x 0.66904654355729
END
